## Agents and Tools

### Agents: Agents enable LLMs to work with tools that perform various tasks
To use agents, we require three things:
- A base LLM,
- A tool that we will be interacting with,
- An agent to control the interaction.

### In the following example, we will use the Math tool to perform some simple math operations

In [1]:
from langchain_openai import OpenAI

In [8]:
llm = OpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo-instruct"
)

In [9]:
from langchain.chains import LLMMathChain # Chain that interprets a prompt and executes python code to do math.
from langchain.agents import Tool

In [10]:
# initialize the math tool

llm_math = LLMMathChain.from_llm(llm=llm)

math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)

# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]  # is a list of possible Tools that your agent can use

In [11]:
print(tools[0].name, "---", tools[0].description)

Calculator --- Useful for when you need to answer questions about math.


## Now create an Agent
#### Zero-shot ReAct
The Zero-shot ReAct Agent is a language generation model that can create realistic contexts even without being trained on specific data. It can be used for various tasks such as generating creative text formats, language translation, and generating different types of creative content.

In [12]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="chat-zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True
)

In [13]:
problem = """
        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen 
        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. 
        What is the total area of your house?
        """
zero_shot_agent(problem)



> Entering new AgentExecutor chain...
Could not parse LLM output: Thought: To find the total area of the house, we need to calculate the area of each room and then add them together.
Action:
{
  "action": "Calculator",
  "action_input": "5 * 5 + 5 * 5 + 7 * 6 + 3 * 2"
}


Observation: Invalid or incomplete response
Thought:Could not parse LLM output:  I need to specify the correct tool to use, which is the Calculator.
Action:
{
  "action": "Calculator",
  "action_input": "5 * 5 + 5 * 5 + 7 * 6 + 3 * 2"
}


Observation: Invalid or incomplete response
Thought:Could not parse LLM output:  I need to specify the correct input for the Calculator tool.
Action:
{
  "action": "Calculator",
  "action_input": "5 * 5 + 5 * 5 + 7 * 6 + 3 * 2"
}



Observation: Invalid or incomplete response
Thought: I now know the final answer
Final Answer: 89

> Finished chain.


{'input': '\n        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen \n        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. \n        What is the total area of your house?\n        ',
 'output': '89'}

## Create a custom tool

In [14]:
from langchain.tools import tool
from langchain_core.prompts import PromptTemplate

@tool("JSON_Response", return_direct=True)
def StructuredResponseTool(question: str):
    """
    use this tool to send a prompt and get a JSON returned 
    with three fields - Topic, Question_Details and Detailed_Response
    """
    json_prompt = PromptTemplate.from_template(
    """Return a JSON object with an `answer` key that answers the following question: {question}. 
        The JSON object will have three fields - Topic, Question_Details and Detailed_Response"""
    )

    model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)

    json_chain = json_prompt | model
    x = json_chain.invoke({"question": question})
    return x

In [15]:
from langchain.agents import initialize_agent

tools = [StructuredResponseTool]

# initialize agent with tools
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

In [16]:
import json

x = zero_shot_agent.run(input="Which is the top football team in Argentina and how, the response should have three fields in the JSON - topic of the question, the question and the detailed response.")

y =json.loads(x)

y

/home/miller/.local/share/virtualenvs/langchain_open_rag-FDDFjH_q/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use the JSON_Response tool to get the answer
Action: JSON_Response
Action Input: "Which is the top football team in Argentina and how"
Observation: 

{
    "Topic": "Football Teams in Argentina",
    "Question_Details": "Which is the top football team in Argentina and how?",
    "Detailed_Response": "The top football team in Argentina is Boca Juniors. They have won the Argentine Primera División 34 times, making them the most successful team in the league. Boca Juniors also has a large and passionate fan base, known as La Doce, who provide unwavering support for the team. Additionally, Boca Juniors has a long history of producing talented players, such as Diego Maradona and Carlos Tevez, who have helped the team achieve success both domestically and internationally."
} 


> Finished chain.


{'Topic': 'Football Teams in Argentina',
 'Question_Details': 'Which is the top football team in Argentina and how?',
 'Detailed_Response': 'The top football team in Argentina is Boca Juniors. They have won the Argentine Primera División 34 times, making them the most successful team in the league. Boca Juniors also has a large and passionate fan base, known as La Doce, who provide unwavering support for the team. Additionally, Boca Juniors has a long history of producing talented players, such as Diego Maradona and Carlos Tevez, who have helped the team achieve success both domestically and internationally.'}